In [190]:
import gsampler as gs
import numpy as np
import pandas as pd
from light.util import selectDates
from IPython.display import display
import time

### Using multithreading

In [191]:
import multiprocessing
import multiprocessing.pool
NUM_THREADS = 8
class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

In [192]:
symbol ='TX_G0.TWF'
colo = 'twf'
dates = selectDates('20171101~20171204')
TICK_SIZE = 0.01
print len(dates)

24


In [193]:
def get_raw_features(symbol, colo, date):
    features = []
    for i in range(2):
        features.append(gs.FeatureAskPrice(symbol = symbol, level = i))
        features.append(gs.FeatureAskQty(symbol = symbol, level = i))
    for i in range(2):
        features.append(gs.FeatureBidPrice(symbol = symbol, level = i))
        features.append(gs.FeatureBidQty(symbol = symbol, level = i))
    features.append(gs.FeatureTradePrice(symbol = symbol))
    features.append(gs.FeatureTradeSide(symbol = symbol))
    features.append(gs.FeatureTradeSize(symbol = symbol))
    features.append(gs.FeatureTickerIdCast())
    sampler = gs.SamplerRunner(features = features, 
                               dates = [date], 
                               colo = colo, 
                               cb_version = 'v2')
    sampler.rerun()
    df = sampler.load_data_df()
    df = df.between_time(start_time="09:00:00", end_time="13:30:00")
    df = df.fillna(0.)
    df.columns = ['ap0', 'az0', 'ap1', 'az1',
                  'bp0', 'bz0', 'bp1', 'bz1',
                  'tp', 'ts', 'tv', 'a', 'b', 'time']
    df.drop('a', axis=1, inplace=True)
    df.drop('b', axis=1, inplace=True)
    df.drop('time', axis=1, inplace=True)
    return df
def wrapper_get_raw_features(args):
    return get_raw_features(*args)

In [194]:
start_time = time.time()
df_test = get_raw_features(symbol=symbol, colo=colo, date=dates[0])
print "Sampling time: ", time.time() - start_time
display(df_test[:])
print df_test.shape
print df_test[df_test.ts!=0].shape

DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/3ef640264d61eacb62fe5c4d0b4b482f5a17c6b5.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/3ef640264d61eacb62fe5c4d0b4b482f5a17c6b5.json
*** Sampling 1/1 <twf> 2:00~16:00 TX_G0.TWF x14  
Starting to work ...
+++ (0:00:08.907864)  /share/ginkgo/bin/current/gsampler.exe -c /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/3ef640264d61eacb62fe5c4d0b4b482f5a17c6b5.json -C market_data.files=/earth/ginkgo/market_data/20171101/kgi_kgi_tk2/kgi_kgi_tk2.twf.20171101.channel_TXFutures.raw.fmsg.gz -C sampler.date=20171101  

* runtime: 0:00:08.918090
Work finished
reading date=20171101 started
Sampling time:  10.3541300297


,ap0,az0,ap1,az1,bp0,bz0,bp1,bz1,tp,ts,tv
time,,,,,,,,,,,
2017-11-01 09:00:00.013070,10823.0,28.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0
2017-11-01 09:00:00.138176,10823.0,28.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,10823.0,1.0,2.0
2017-11-01 09:00:00.138176,10823.0,26.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0
2017-11-01 09:00:00.138187,10823.0,26.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,10823.0,1.0,1.0
2017-11-01 09:00:00.138187,10823.0,25.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0
2017-11-01 09:00:00.138235,10823.0,25.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,10822.0,-1.0,6.0
2017-11-01 09:00:00.138235,10823.0,25.0,10824.0,56.0,10821.0,37.0,10820.0,69.0,0.0,0.0,0.0
2017-11-01 09:00:00.138260,10823.0,42.0,10824.0,56.0,10822.0,5.0,10821.0,38.0,0.0,0.0,0.0
2017-11-01 09:00:00.263096,10823.0,42.0,10824.0,56.0,10822.0,5.0,10821.0,38.0,10823.0,1.0,27.0


(153406, 11)
(29801, 11)


In [195]:
args = []
for date in dates:
    args.append((symbol, colo, date))

In [196]:
start_time = time.time()
pool = MyPool(NUM_THREADS)
dfs = pool.map(wrapper_get_raw_features, args)
pool.close()
pool.join()
print "Sampling time: ", time.time() - start_time

DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/d4138bc1b7e316dcaf7deb122fbef6ecd85cce04.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/df819be46a8ef5ae190b7172f2d1bb6258b8e9ec.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/3ef640264d61eacb62fe5c4d0b4b482f5a17c6b5.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/7e6a615e7ecdf4a0f6a145626a5e4ab963b893b0.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/e3972d9371c6b72c46893883605a860137f9438f.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/57081c71a6604d945b6fb94e9b40a09ae33998ab.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/5dd


* runtime: 0:00:13.443677
Work finished
Starting to work ...
Starting to work ...
reading date=20171108 started
reading date=20171109 started
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/7aafce5b3b2c7c6e3a0b3fe6aa2715e37c77d8c5.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/7aafce5b3b2c7c6e3a0b3fe6aa2715e37c77d8c5.json
*** Sampling 1/1 <twf> 2:00~16:00 TX_G0.TWF x14  
Starting to work ...
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/9d7eaa73e1b6666a2378afc4c2c6a3225fa789c2.json
DEBUG(PY: /ginkgo/Json.py:128:save) Writing file: /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/9d7eaa73e1b6666a2378afc4c2c6a3225fa789c2.json
*** Sampling 1/1 <twf> 2:00~16:00 TX_G0.TWF x14  
Starting to work ...
+++ (0:00:11.106385)  /share/ginkgo/bin/current/gsampler.exe -c /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/ab

Work finished
reading date=20171127 started
+++ (0:00:11.671925)  /share/ginkgo/bin/current/gsampler.exe -c /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/e9983f29674da33de81ec78dc66aa7b9d078348f.json -C market_data.files=/earth/ginkgo/market_data/20171128/kgi_kgi_tk2/kgi_kgi_tk2.twf.20171128.channel_TXFutures.raw.fmsg.gz -C sampler.date=20171128  


* runtime: 0:00:11.678023
Work finished+++ (0:00:11.723528)  /share/ginkgo/bin/current/gsampler.exe -c /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/088d72fd6984799ba7d0be330da0c7217945c593.json -C market_data.files=/earth/ginkgo/market_data/20171129/kgi_kgi_tk2/kgi_kgi_tk2.twf.20171129.channel_TXFutures.raw.fmsg.gz -C sampler.date=20171129  

* runtime: 0:00:11.729742
Work finished
reading date=20171128 started
reading date=20171129 started
+++ (0:00:12.395761)  /share/ginkgo/bin/current/gsampler.exe -c /cache/zebo/bee21fd3fd53c7e769e394e61beb15510c0ec694/e49fb578fd78ebbb79ced53c2198a67e0aadcf1c.json -C market_data.files=/ear

In [197]:
for df in dfs:
    for vec in df.values:
        if abs(vec[0] + 1.- vec[2]) > 0.0001:
            print vec
            break
        if abs(vec[6] + 1.- vec[4]) > 0.0001:
            print vec
            break

[ 10783.     64.      0.      0.  10778.     23.  10777.     48.      0.
      0.      0.]
[ 10820.     60.      0.      0.  10815.     17.  10814.     58.      0.
      0.      0.]
[  1.06360000e+04   1.00000000e+00   1.06380000e+04   1.70000000e+01
   1.06350000e+04   3.00000000e+00   1.06340000e+04   1.90000000e+01
   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[  1.06820000e+04   5.00000000e+00   1.06830000e+04   3.90000000e+01
   1.06770000e+04   7.80000000e+01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[  1.08200000e+04   1.00000000e+00   0.00000000e+00   0.00000000e+00
   1.08140000e+04   2.00000000e+00   1.08130000e+04   1.90000000e+01
   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[ 10601.     22.  10602.     68.  10596.     35.      0.      0.      0.
      0.      0.]


##### Train other people's stgy
- Send an aggressive buy order (0)
- Send an aggressive sell order (1)
- Send Market making buy order (2)
- Send Market making sell order (3)
- Send a passive buy order (4)
- Send a passive sell order (5)
- Cancel a passive buy order (6)
- Cancel a passive sell order (7)
- Do nothing (8)

In [242]:
def getLabels(prev,post):
    if prev[9] != 0:
        return None
    if post[9] == 1:
        return [0]
    if post[9] == -1:
        return [1]
    target =[]
    if prev[0] != post[0] or prev[4] != post[4]:
        if post[0] > prev[0]:
            target.append(2)
        elif post[0] < prev[0]:
            target.append(3)
        if post[4] > prev[4]:
            target.append(2)
        elif post[4] < prev[4]:
            target.append(3)
        return list(set(target))
    elif post[1] != prev[1] or post[5] != prev[5]:
        if post[1] > prev[1]:
            target.append(5)
        elif post[1] < prev[1]:
            target.append(7)
        if post[5] > prev[5]:
            target.append(4)
        elif post[5] < prev[5]:
            target.append(6)
        return list(set(target))
    else:
        return [8]
def getLabelsWrapper(args):
    return getLabels(*args)

In [199]:
mat = (df_test.values*(1.00001)).astype(np.int16)
print mat[:10]

[[10823    28 10824    56 10822     3 10821    37     0     0     0]
 [10823    28 10824    56 10822     3 10821    37 10823     1     2]
 [10823    26 10824    56 10822     3 10821    37     0     0     0]
 [10823    26 10824    56 10822     3 10821    37 10823     1     1]
 [10823    25 10824    56 10822     3 10821    37     0     0     0]
 [10823    25 10824    56 10822     3 10821    37 10822    -1     6]
 [10823    25 10824    56 10821    37 10820    69     0     0     0]
 [10823    42 10824    56 10822     5 10821    38     0     0     0]
 [10823    42 10824    56 10822     5 10821    38 10823     1    27]
 [10823    15 10824    56 10822     5 10821    38     0     0     0]]


In [200]:
args = [(mat[i-1], mat[i]) for i in range(1, len(mat))]
print args[:3]

[(array([10823,    28, 10824,    56, 10822,     3, 10821,    37,     0,
           0,     0], dtype=int16), array([10823,    28, 10824,    56, 10822,     3, 10821,    37, 10823,
           1,     2], dtype=int16)), (array([10823,    28, 10824,    56, 10822,     3, 10821,    37, 10823,
           1,     2], dtype=int16), array([10823,    26, 10824,    56, 10822,     3, 10821,    37,     0,
           0,     0], dtype=int16)), (array([10823,    26, 10824,    56, 10822,     3, 10821,    37,     0,
           0,     0], dtype=int16), array([10823,    26, 10824,    56, 10822,     3, 10821,    37, 10823,
           1,     1], dtype=int16))]


In [201]:
start_time = time.time()
pool = MyPool(NUM_THREADS)
targets = pool.map(getLabelsWrapper, args)
pool.close()
pool.join()
print "Sampling time: ", time.time() - start_time
print targets[:20]

Sampling time:  3.54483413696
[[0], -1, [0], -1, [1], -1, [2], [0], -1, [1], -1, [0], -1, [0], -1, [4, 5], [0], -1, [5, 6], [1]]


In [202]:
print len(mat), len(targets)

153406 153405


In [203]:
targets.append(None)

In [204]:
df_test.insert(len(df_test.columns), 'target', pd.Series(targets, index=df_test.index))
display(df_test[:5])

,ap0,az0,ap1,az1,bp0,bz0,bp1,bz1,tp,ts,tv,target
time,,,,,,,,,,,,
2017-11-01 09:00:00.013070,10823.0,28.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0,[0]
2017-11-01 09:00:00.138176,10823.0,28.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,10823.0,1.0,2.0,-1
2017-11-01 09:00:00.138176,10823.0,26.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0,[0]
2017-11-01 09:00:00.138187,10823.0,26.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,10823.0,1.0,1.0,-1
2017-11-01 09:00:00.138187,10823.0,25.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0,[1]


In [205]:
df_test = df_test[df_test.target != -1]
display(df_test[:5])

,ap0,az0,ap1,az1,bp0,bz0,bp1,bz1,tp,ts,tv,target
time,,,,,,,,,,,,
2017-11-01 09:00:00.013070,10823.0,28.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0,[0]
2017-11-01 09:00:00.138176,10823.0,26.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0,[0]
2017-11-01 09:00:00.138187,10823.0,25.0,10824.0,56.0,10822.0,3.0,10821.0,37.0,0.0,0.0,0.0,[1]
2017-11-01 09:00:00.138235,10823.0,25.0,10824.0,56.0,10821.0,37.0,10820.0,69.0,0.0,0.0,0.0,[2]
2017-11-01 09:00:00.138260,10823.0,42.0,10824.0,56.0,10822.0,5.0,10821.0,38.0,0.0,0.0,0.0,[0]


##### Look Back = 6
Generating features

In [212]:
look_back = 6
num_levels = 10
tick_size = 1.
def getImage(vecs):
    assert len(vecs) == look_back
    img = np.zeros((look_back, num_levels, 4))
    ask_base = vecs[5][0] - tick_size*num_levels/2
    bid_base = vecs[5][4] - tick_size*(num_levels/2 - 1)
    for i in range(look_back):
        l = int((vecs[i][0] + 0.001 - ask_base)/tick_size)
        if l>=0 and l<num_levels:
            img[i][l][0] = vecs[i][1]
        l = int((vecs[i][2] + 0.001 - ask_base)/tick_size)
        if l>=0 and l<num_levels:
            img[i][l][0] = vecs[i][3]
        l = int((vecs[i][4] + 0.001 - bid_base)/tick_size)
        if l>=0 and l<num_levels:
            img[i][l][1] = vecs[i][5]
        l = int((vecs[i][6] + 0.001 - bid_base)/tick_size)
        if l>=0 and l<num_levels:
            img[i][l][1] = vecs[i][7]
        if vecs[i][9] > 0.5:
            l = int((vecs[i][8] + 0.001 - ask_base)/tick_size)
            if l>=0 and l<num_levels:
                img[i][l][2] = vecs[i][10]
        if vecs[i][9] < -0.5:
            l = int((vecs[i][8] + 0.001 - bid_base)/tick_size)
            if l>=0 and l<num_levels:
                img[i][l][3] = vecs[i][10]
    return img

In [214]:
arg = np.array(mat[:6])
print arg
A = getImage(arg)
print A.shape

[[10823    28 10824    56 10822     3 10821    37     0     0     0]
 [10823    28 10824    56 10822     3 10821    37 10823     1     2]
 [10823    26 10824    56 10822     3 10821    37     0     0     0]
 [10823    26 10824    56 10822     3 10821    37 10823     1     1]
 [10823    25 10824    56 10822     3 10821    37     0     0     0]
 [10823    25 10824    56 10822     3 10821    37 10822    -1     6]]
(6, 10, 4)


In [218]:
print A[:,:,0],'\n', A[:,:,1],'\n', A[:,:,2],'\n', A[:,:,3]

[[  0.   0.   0.   0.   0.  28.  56.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  28.  56.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  26.  56.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  26.  56.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  25.  56.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  25.  56.   0.   0.   0.]] 
[[  0.   0.   0.  37.   3.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  37.   3.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  37.   3.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  37.   3.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  37.   3.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  37.   3.   0.   0.   0.   0.   0.]] 
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  2.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]] 
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 

In [230]:
args = [np.array(mat[i:i+6]) for i in range(len(mat) - 10)]
for arg in args:
    if arg.shape != (6, 11):
        print arg
start_time = time.time()
pool = MyPool(NUM_THREADS)
imgs = np.array(pool.map(getImage, args))
pool.close()
pool.join()
print "Sampling time: ", time.time() - start_time
print imgs.shape

Sampling time:  4.84107398987
(153396, 6, 10, 4)


In [231]:
def _getRnnAskPrice(vecs):
    assert len(vecs) == look_back
    return [vec[0] for vec in vecs]
def _getRnnBidPrice(vecs):
    assert len(vecs) == look_back
    return [vec[4] for vec in vecs]

In [232]:
start_time = time.time()
pool = MyPool(NUM_THREADS)
asks = np.array(pool.map(_getRnnAskPrice, args))
pool.close()
pool.join()
print "Sampling time: ", time.time() - start_time
print asks.shape

Sampling time:  8.15105319023
(153396, 6)


In [233]:
start_time = time.time()
pool = MyPool(NUM_THREADS)
bids = np.array(pool.map(_getRnnBidPrice, args))
pool.close()
pool.join()
print "Sampling time: ", time.time() - start_time
print bids.shape

Sampling time:  5.66164684296
(153396, 6)


In [234]:
print asks[0]
print bids[0]

[10823 10823 10823 10823 10823 10823]
[10822 10822 10822 10822 10822 10822]


In [243]:
args = [(mat[i+6], mat[i+7]) for i in range(len(mat) - 10)]
start_time = time.time()
pool = MyPool(NUM_THREADS)
targets = pool.map(getLabelsWrapper, args)
pool.close()
pool.join()
print "Sampling time: ", time.time() - start_time
print len(targets)

Sampling time:  2.89680695534
153396


##### Parsing features

In [245]:
f_asks = []
f_bids = []
f_imgs = []
f_targes = []
for i in range(len(targets)):
    if targets[i] is None:
        continue
    for x in targets[i]:
        f_asks.append(np.array(asks[i]))
        f_bids.append(np.array(bids[i]))
        f_imgs.append(np.array(imgs[i]))
        f_targes.append(int(x))
print np.array(f_asks).shape, np.array(f_bids).shape, np.array(f_imgs).shape, np.array(f_targes).shape

(134710, 6) (134710, 6) (134710, 6, 10, 4) (134710,)


In [246]:
for x in set(f_targes):
    print x, f_targes.count(x)

0 14621
1 14773
2 2183
3 2225
4 16662
5 17447
6 9624
7 9865
8 47310
